安装opencompass：Kaggle上已经为我们准备好了其他常用包，只需安装opencompass用于评测即可。如果不在Kaggle上运行，则还需要安装其他必要包。

In [1]:
pwd

'/kaggle/working'

In [2]:
!pip install "opencompass[full]"
# !pip install pytorch transformers datasets "opencompass[full]"

In [3]:
pip uninstall transformers --yes

In [4]:
pip install transformers

# 指令微调

In [5]:
"""
The main program for finetuning LLMs with Huggingface Transformers Library.

ALL SECTIONS WHERE CODE POSSIBLY NEEDS TO BE FILLED IN ARE MARKED AS TODO.
"""

import argparse
from dataclasses import dataclass, field
from typing import Optional, List, Dict
import sys
import torch
from transformers import TrainingArguments, HfArgumentParser, Trainer, AutoTokenizer, AutoModelForCausalLM
import datasets
print("done")

done


In [6]:
# Define the arguments required for the main program.
# NOTE: You can customize any arguments you need to pass in.
@dataclass
class ModelArguments:
    """Arguments for model
    """
    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The path to the LLM to fine-tune or its name on the Hugging Face Hub."
        }
    )
    torch_dtype: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "Override the default `torch.dtype` and load the model under this dtype."
            ),
            "choices": ["bfloat16", "float16", "float32"],
        },
    )
    # TODO: add your model arguments here
    pass


@dataclass
class DataArguments:
    """Arguments for data
    """
    dataset_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The path to the fine-tuning dataset or its name on the Hugging Face Hub."
        }
    )
    # TODO: add your data arguments here
    
@dataclass
class TrainingArguments(TrainingArguments):
    """Arguments for training
    """
    output_dir: str = field(
        default=None
    )
    per_device_train_batch_size: Optional[int] = field(
        default=1
    )
    save_strategy: Optional[str] = field(
        default="steps"
    )
    save_steps: Optional[int] = field(
        default=3000
    )
    save_total_limit: Optional[int] = field(
        default=1
    )
    report_to: Optional[str] = field(
        default="tensorboard"
    )
    warmup_ratio: Optional[float] = field(
        default=0.03
    )
    logging_steps: Optional[int] = field(
        default=100
    )
    per_device_eval_batch_size: Optional[int] = field(
        default=1
    )
    num_train_epochs: Optional[int] = field(
        default=2
    )
    learning_rate: Optional[float] = field(
        default=1e-5
    )
    optim: Optional[str] = field(
        default="adamw_hf"
    )
    lr_scheduler_type: Optional[str] = field(
        default="cosine"
    )
    weight_decay: Optional[float] = field(
        default=0.001
    )
    print("done")

done


In [7]:
# The main function
# NOTE You can customize some logs to monitor your program.
def finetune():
    # TODO Step 1: Define an arguments parser and parse the arguments
    # NOTE Three parts: model arguments, data arguments, and training arguments
    # HINT: Refer to 
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/internal/trainer_utils#transformers.HfArgumentParser
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/main_classes/trainer#transformers.TrainingArguments
    parser = HfArgumentParser((ModelArguments, DataArguments, TrainingArguments))
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()
    print("step1 done")
    print(training_args)

    # TODO Step 2: Load tokenizer and model
    # HINT 1: Refer to
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/main_classes/tokenizer#tokenizer
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/model_doc/qwen2
    # HINT 2: To save training GPU memory, you need to set the model's parameter precision to half-precision (float16 or bfloat16).
    #         You may also check other strategies to save the memory!
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/model_doc/llama2#usage-tips
    #   * https://huggingface.co/docs/transformers/perf_train_gpu_one
    #   * https://www.53ai.com/news/qianyanjishu/2024052494875.html
    tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path)
    model = AutoModelForCausalLM.from_pretrained(model_args.model_name_or_path, torch_dtype=model_args.torch_dtype, device_map="auto")
    print("step2 done")
    
    # TODO Step 3: Load dataset
    # HINT: https://huggingface.co/docs/datasets/v3.1.0/en/package_reference/main_classes#datasets.Dataset
    dataset = datasets.load_dataset(data_args.dataset_path)
    dataset = dataset["train"]
    print("step3 done")
    
    # TODO Step 4: Define the data collator function
    # NOTE During training, for each model parameter update, we fetch a batch of data, perform a forward and backward pass,
    # and then update the model parameters. The role of the data collator is to process the data (e.g., padding the data within
    # a batch to the same length) and format the batch into the input required by the model.
    #
    # In this assignment, the purpose of the custom data_collator is to process each batch of data from the dataset loaded in
    # Step 3 into the format required by the model. This includes tasks such as tokenizing the data, converting each token into 
    # an ID sequence, applying padding, and preparing labels.
    # 
    # HINT:
    #   * Before implementation, you should:
    #      1. Clearly understand the format of each sample in the dataset loaded in Step 3.
    #      2. Understand the input format required by the model (https://huggingface.co/docs/transformers/model_doc/qwen2#transformers.Qwen2ForCausalLM).
    #         Reading its source code also helps!
    padding = -100
    def preprocess(dataset, max_len=1024):
        question_template = '<|im_start|>system\nYou are a knowledgeable assistant. Please give a detailed response to the following question.<|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n'
        prompt_template = '<|im_start|>system\nYou are a knowledgeable assistant. Please give a detailed response to the following question.<|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n{response}<im_end>' #定义训练时用的prompt
        model_inputs = {'input_ids': [], 'attention_mask':[], 'labels': []} #初始化返回的字典
        for i in range(len(dataset['instruction'])): #遍历
            instruct = dataset['instruction'][i] if dataset['instruction'][i] else '' # 有些数据的instruct和input没有内容，为None
            input = dataset['input'][i] if dataset['input'][i] else ''
            question = instruct+ '\n' + input
            prompt = prompt_template.format(
                        question=question,
                        response=dataset['output'][i],
                    ) #获得prompt
            tokenized_id = tokenizer(
                prompt,
                return_tensors="pt",
                padding="max_length",
                truncation=True,
                max_length=max_len
            ) #tokenizing the data
            labels = tokenized_id['input_ids'].clone()
            labels[labels==tokenizer.pad_token_id] = padding #用-100padding填充使CrossEntropyLoss忽略填充的部分，只计算非填充部分的loss
            labels = labels.flatten()
            input_id = tokenized_id['input_ids'].flatten()
            attention_mask = tokenized_id['attention_mask'].flatten() #获得输出字典中的input_ids, attention_mask和labels
            #如果只算output部分
            #response_start = len(tokenizer.encode(question_template.format(question=dataset['instruction'][i])))
            #if response_start>max_len:
            #    response_start = max_len
            #labels[:response_start] = padding
                
            model_inputs['input_ids'].append(input_id)
            model_inputs['attention_mask'].append(attention_mask)
            model_inputs['labels'].append(labels)
        return model_inputs
    
    dataset = dataset.map(preprocess, batched=True, num_proc=1, load_from_cache_file=False) #用提前对dataset进行map操作替代custom data_collator，节约后续训练时间。
    print("step4 done")
    
    def data_collator(batch: List[Dict]):
        """
        batch: list of dict, each dict of the list is a sample in the dataset.
        """
        pass
    
    # TODO Step 5: Define the Trainer
    # HINT: https://huggingface.co/docs/transformers/main_classes/trainer
    trainer = Trainer(
        train_dataset=dataset,
        #eval_dataset=dataset["test"], 不进行训练测试集切分，而使用后续的数据集进行评估
        tokenizer=tokenizer,
        args=training_args,
        model=model,
    )
    print("step5 done")
    # Step 6: Train!
    trainer.train()
print("done")

done


In [8]:
# Pass your training arguments.
# NOTE [IMPORTANT!!!] DO NOT FORGET TO PASS PROPER ARGUMENTS TO SAVE YOUR CHECKPOINTS!!!
sys.argv = [
    "notebook", 
    "--model_name_or_path", "/kaggle/input/qwen2.5/transformers/0.5b/1",
    "--torch_dtype", "bfloat16", #使用bfloat16来减小显存需求
    "--dataset_path", "/kaggle/input/alpaca-language-instruction-training",
    "--output_dir", "/kaggle/working/output",
    "--per_device_train_batch_size", "1", #16G显存使用4会爆显存
    "--save_strategy", "steps",
    "--save_steps", "1000",
    "--save_total_limit", "1",
    "--report_to", "tensorboard",
    "--warmup_ratio", "0.03",
    "--logging_steps", "100",
    "--num_train_epochs", "1",
    "--learning_rate", "1e-5",
    "--optim", "adamw_hf",
    "--weight_decay", "1e-3",
    "--lr_scheduler_type", "cosine",
    #'--max_steps', '6000'
]
finetune()

step1 done
TrainingArguments(
_n_gpu=2,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=no,
eval_use_gath

step2 done


Generating train split: 0 examples [00:00, ? examples/s]

step3 done


Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

step4 done


/tmp/ipykernel_23/4079481066.py:88: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


step5 done


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
100,3.075900
200,2.703300
300,2.285200
400,1.897200
500,1.699400
600,1.564000
700,1.535200
800,1.537600
900,1.398800
1000,1.423600


# 评测模型

In [ ]:

PLM_MODEL_PATH = "/kaggle/input/qwen2.5/transformers/0.5b/1"
SFT_MODEL_PATH = "/kaggle/working/output/checkpoint-30000"


如果你有多个GPU，可以修改下面的--hf-num-gpus参数来加速评测。

In [ ]:

!opencompass \
    --datasets mmlu_ppl hellaswag_clean_ppl winogrande_ll ARC_e_ppl ARC_c_clean_ppl SuperGLUE_BoolQ_few_shot_ppl \
    --summarizer example \
    --hf-type base \
    --hf-path {PLM_MODEL_PATH} \
    --tokenizer-kwargs padding_side="left" truncation="left" \
    --max-seq-len 1024 \
    --batch-size 1 \
    --hf-num-gpus 2 \
    --work-dir "/kaggle/working/evals/plm" \
    --debug


In [ ]:

!opencompass \
    --datasets mmlu_ppl hellaswag_clean_ppl winogrande_ll ARC_e_ppl ARC_c_clean_ppl SuperGLUE_BoolQ_few_shot_ppl \
    --summarizer example \
    --hf-type base \
    --hf-path {SFT_MODEL_PATH} \
    --tokenizer-kwargs padding_side="left" truncation="left" \
    --max-seq-len 1024 \
    --batch-size 1 \
    --hf-num-gpus 2 \
    --work-dir "/kaggle/working/evals/sft" \
    --debug


In [ ]:
#使用原始模型进行生成

sys.argv = [
    "notebook", 
    "--model_name_or_path", "/kaggle/input/qwen2.5/transformers/0.5b/1",
    "--torch_dtype", "bfloat16", #使用bfloat16来减小显存需求
    "--dataset_path", "/kaggle/input/alpaca-language-instruction-training",
    "--output_dir", "/kaggle/working/output",
    "--per_device_train_batch_size", "1", #16G显存使用4会爆显存
    "--save_strategy", "steps",
    "--save_steps", "1000",
    "--save_total_limit", "1",
    "--report_to", "tensorboard",
    "--warmup_ratio", "0.03",
    "--logging_steps", "100",
    "--num_train_epochs", "1",
    "--learning_rate", "1e-5",
    "--optim", "adamw_hf",
    "--weight_decay", "1e-3",
    "--lr_scheduler_type", "cosine",
]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
parser = HfArgumentParser((ModelArguments, DataArguments, TrainingArguments))
model_args, data_args, training_args = parser.parse_args_into_dataclasses()
tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(model_args.model_name_or_path, torch_dtype=model_args.torch_dtype, device_map="auto")
input_text = '<|im_start|>system\nYou are a knowledgeable assistant. Please give a detailed response to the following question.<|im_end|>\n<|im_start|>user\n{Introduce Natural Language Processing(NLP)\n}<|im_end|>\n<|im_start|>assistant\n'
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
outputs = model.generate(input_ids["input_ids"], max_new_tokens=500)
decoded_outputs = tokenizer.decode(outputs[0])
print(f"Result: {decoded_outputs}")


In [ ]:
#使用微调后的模型进行生成

sys.argv = [
    "notebook", 
    "--model_name_or_path", "/kaggle/working/output/checkpoint-30000",
    "--torch_dtype", "bfloat16", #使用bfloat16来减小显存需求
    "--dataset_path", "/kaggle/input/alpaca-language-instruction-training",
    "--output_dir", "/kaggle/working/output",
    "--per_device_train_batch_size", "1",
    "--save_strategy", "steps",
    "--save_steps", "1000",
    "--save_total_limit", "1",
    "--report_to", "tensorboard",
    "--warmup_ratio", "0.03",
    "--logging_steps", "100",
    "--num_train_epochs", "1",
    "--learning_rate", "1e-5",
    "--optim", "adamw_hf",
    "--weight_decay", "1e-3",
    "--lr_scheduler_type", "cosine",
]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
parser = HfArgumentParser((ModelArguments, DataArguments, TrainingArguments))
model_args, data_args, training_args = parser.parse_args_into_dataclasses()
tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(model_args.model_name_or_path, torch_dtype=model_args.torch_dtype, device_map="auto")
input_text = '<|im_start|>system\nYou are a knowledgeable assistant. Please give a detailed response to the following question.<|im_end|>\n<|im_start|>user\n{Introduce Natural Language Processing(NLP)\n}<|im_end|>\n<|im_start|>assistant\n'
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
outputs = model.generate(input_ids["input_ids"], max_new_tokens=500)
decoded_outputs = tokenizer.decode(outputs[0])
print(f"Result: {decoded_outputs}")
